## Overview

This notebook:

- creates a MessageHub instance in Bluemix
- produces some messages 
- consumes those messages
- tears down the MessageHub instance

Note that this demo does not use spark python - it just uses vanilla python.

Enter your bluemix ID

In [3]:
from getpass import getpass
ibm_id = raw_input("IBM ID: ")

IBM ID: chris.snow@uk.ibm.com


Enter your bluemix ID password

In [4]:
from getpass import getpass
ibm_id_password = getpass("Password: ")

Password: ········


Setup some variables that this notebook will use

In [5]:
# change this to point to your bluemix organization name
bluemix_organization_name = 'chris.snow@uk.ibm.com'

# change this to point to your bluemix space name
bluemix_space_name = 'dev'

# You may need to change the target_endpoint to:
#
#   https://api.ng.bluemix.net     - for the US South Region
#   https://api.eu-gb.bluemix.net  - for the UK
#   https://api.au-syd.bluemix.net - for Austrailia
target_endpoint = 'https://api.ng.bluemix.net'

# This is the name of the Message Hub service instance that will
# be created for you in Bluemix
messagehub_instance_name = 'my_messagehub'

# This is the name of a topic that will get created for you
messagehub_topic_name = 'my_topic'

# Do you want to delete any existing MessageHub instance with
# messagehub_instance_name for before creating a new one?
delete_messagehub_instance = True

# This is the guid of the Message Hub service plan. I found
# this using:
#
#   cf = CloudFoundryUtil(target_endpoint, ibm_id, ibm_id_password)
#   print(cf.search_plans('message hub'))
mh_service_plan_guid = 'fe959ac5-aa47-43a6-9c58-6fc265ee9b0e'

Install a python utility script from github for interacting with cloud foundry via the API

In [8]:
!pip install --user --upgrade protobuf
!pip install --user --upgrade --quiet git+https://github.com/snowch/cf_utils

Create a python object for interacting with cloud foundry

In [9]:
from cf_utils import cf_utils
cf = cf_utils.CloudFoundryUtil(target_endpoint, ibm_id, ibm_id_password)

Remove any MessageHub instances with the messagehub_instance_name

In [10]:
if delete_messagehub_instance:
    cf.delete_service(service_instance_name=messagehub_instance_name, force=True)

Create a MessageHub service instance

In [11]:
cf.create_service_instance(
    bluemix_organization_name, 
    bluemix_space_name,
    mh_service_plan_guid,
    messagehub_instance_name,
)

Extract some variables that we will need to work with the MessageHub instance

In [12]:
bootstrap_servers = cf.get_service_credential(messagehub_instance_name, 'kafka_brokers_sasl')
# print(bootstrap_servers)

sasl_username = cf.get_service_credential(messagehub_instance_name, 'user')
# print(sasl_username)

sasl_password = cf.get_service_credential(messagehub_instance_name, 'password')
# print(sasl_password)

api_key = cf.get_service_credential(messagehub_instance_name, 'api_key')
# print(api_key)

kafka_admin_url = cf.get_service_credential(messagehub_instance_name, 'kafka_admin_url')
# print(kafka_admin_url)

Create a topic in the MessageHub instance

In [20]:
import requests
import json

data = { 'name' : messagehub_topic_name }
headers = {
    'content-type': 'application/json',
    'X-Auth-Token' : api_key 
}

response = requests.post(
                kafka_admin_url + '/admin/topics', 
                headers = headers,
                data = json.dumps(data)
            )

Verify the topic was created ok

In [14]:
import requests
import json

data = { 'name' : messagehub_topic_name }
headers = {
    'content-type': 'application/json',
    'X-Auth-Token' : api_key 
}

response = requests.get(
                kafka_admin_url + '/admin/topics', 
                headers = headers,
                data = json.dumps(data)
            )
print (response.text)

[{"name":"my_topic","partitions":1,"retentionMs":"86400000","markedForDeletion":false}]


Install a python kafka client

In [15]:
!pip install --user --quiet kafka-python

Create a kafka producer and send some messages

In [16]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import ssl

sasl_plain_username = sasl_username
sasl_plain_password = sasl_password 

sasl_mechanism = 'PLAIN'
security_protocol = 'SASL_SSL'

# Create a new context using system defaults, disable all but TLS1.2
context = ssl.create_default_context()
context.options &= ssl.OP_NO_TLSv1
context.options &= ssl.OP_NO_TLSv1_1

producer = KafkaProducer(bootstrap_servers = bootstrap_servers,
                         sasl_plain_username = sasl_plain_username,
                         sasl_plain_password = sasl_plain_password,
                         security_protocol = security_protocol,
                         ssl_context = context,
                         sasl_mechanism = sasl_mechanism,
                         api_version=(0,10))

producer.send(messagehub_topic_name, b'some_raw_bytes_1')
producer.send(messagehub_topic_name, b'some_raw_bytes_2')

producer.flush()

Create a kafka consumer to consume the messages

In [17]:
from kafka import KafkaConsumer
from kafka.errors import KafkaError
import ssl

sasl_plain_username = sasl_username
sasl_plain_password = sasl_password 

sasl_mechanism = 'PLAIN'
security_protocol = 'SASL_SSL'

# Create a new context using system defaults, disable all but TLS1.2
context = ssl.create_default_context()
context.options &= ssl.OP_NO_TLSv1
context.options &= ssl.OP_NO_TLSv1_1

consumer = KafkaConsumer(messagehub_topic_name,
                         bootstrap_servers = bootstrap_servers,
                         sasl_plain_username = sasl_plain_username,
                         sasl_plain_password = sasl_plain_password,
                         security_protocol = security_protocol,
                         ssl_context = context,
                         sasl_mechanism = sasl_mechanism,
                         api_version=(0,10),
                         consumer_timeout_ms=60000,
                         auto_offset_reset='earliest')

for message in consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: `message.value.decode('utf-8')`
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))


my_topic:0:0: key=None value=some_raw_bytes_1
my_topic:0:1: key=None value=some_raw_bytes_2


Remove the MessageHub instance from Bluemix

In [ ]:
if delete_messagehub_instance:
    cf.delete_service(service_instance_name=messagehub_instance_name, force=True)